In [1]:
import math
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt
import numpy as np
import time
import statistics as st
from scipy.spatial import distance as dt
import distance
from operator import truediv, add, mul

### Data loading

In [2]:
def load_Data(name):
    try:
        # TRAIN data
        # load fingerprints
        df_trnrss = pd.read_csv('data/'+ name + '_trnrss.csv', header=None).values.tolist()
        # load coordinates
        df_trncrd = pd.read_csv('data/' + name + '_trncrd.csv', header=None)

        # for now, drop columns of 'floor' and 'building', keep x,y,z
        df_trncrd.drop(df_trncrd.columns[[3,4]], axis=1, inplace=True)
        df_trncrd = df_trncrd.values.tolist()

        # TEST data
        # load fingerprints
        df_tstrss = pd.read_csv('data/' + name + '_tstrss.csv', header=None).values.tolist()
        # load coordinates
        df_tstcrd = pd.read_csv('data/' + name + '_tstcrd.csv', header=None)

        # for now, drop columns of 'floor' and 'building', keep x,y,z
        df_tstcrd.drop(df_tstcrd.columns[[3,4]], axis=1, inplace=True)
        df_tstcrd = df_tstcrd.values.tolist()

        return(df_trnrss, df_trncrd, df_tstrss, df_tstcrd)
    except:
        print("Could not open file")
        return

### Get neighbors

In [3]:
# Locate the most similar neighbors and return list of indexes
def get_neighbors(train, test_row, k):
    distances = list()
    for idx, train_row in enumerate(train):
        # Lp Minkowski family
        dist = math.dist(test_row, train_row) #euclidean
        #dist = dt.minkowski(test_row, train_row, 5)
        #dist = dt.cityblock(test_row, train_row)
        #dist = dt.chebyshev(test_row, train_row)
        
        # L1 family
        #dist = distance.sorensen(test_row, train_row)
        #dist = gower(test_row, train_row)
        #dist = soergel(test_row, train_row)
        #dist = kulczynski_d(test_row, train_row)
        #dist = lorentzian(test_row, train_row)
        #dist = dt.canberra(test_row, train_row)
        
        # Intersection family
        #dist = intersection(test_row, train_row)
        #dist = wavehedges(test_row, train_row)
        #dist = czekanowski_s(test_row, train_row)
        #dist = czekanowski_d(test_row, train_row)
        #dist = motyka_s(test_row, train_row)
        #dist = motyka_d(test_row, train_row)
        #dist = kulczynski_s(test_row, train_row)
        #dist = ruzicka(test_row, train_row)
        #dist = tanimoto(test_row, train_row)
        
        # Inner product family
        #dist = np.inner(test_row, train_row)
        #dist = harmonic(test_row, train_row)
        #dist = dt.cosine(test_row, train_row)
        #dist = kumar(test_row, train_row)
        #dist = jaccard_s(test_row, train_row)
        #dist = jaccard_d(test_row, train_row)
        #dist = dice_s(test_row, train_row)
        #dist = dice_d(test_row, train_row)
        
        # Fidelity family or Squared-chord family
        #dist = fidelity(test_row, train_row)
        #dist = bhattacharrya(test_row, train_row)
        #dist = hellinger(test_row, train_row)
        #dist = matusita(test_row, train_row)
        #dist = squared_chord(test_row, train_row)
        
        # Squared L2 family
        #dist = dt.sqeuclidean(test_row, train_row)
        #dist = pearson(test_row, train_row)
        #dist = neyman(test_row, train_row)
        #dist = squared(test_row, train_row)
        #dist = prob_sym(test_row, train_row)
        #dist = divergence(test_row, train_row)
        #dist = clark(test_row, train_row)
        #dist = additive_sym(test_row, train_row)
        
        # Shannon's entropy family
        #dist = kullback_PQ(test_row, train_row)
        #dist = jeffreys(test_row, train_row)
        #dist = k_divergence(test_row, train_row)
        #dist = topsoe(test_row, train_row)
        #dist = jensen_shannon(test_row, train_row)
        #dist = jensen_diff(test_row, train_row)
        
        # Combinations
        #dist = taneja(test_row, train_row)
        #dist = kumar_johnson(test_row, train_row)
        #dist = avgL(test_row, train_row)
        
        
        distances.append((idx, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = [distances[i][0] for i in range(k)]
    return neighbors

### Knn, predict positions and calculate error

In [8]:
# calculate knn of fingerprints
def knn(k, tstrss, trnrss):
    all_neighbors = []

    # iterate test data and find id of k nearest neighbors from trnrss
    for i in tstrss:
        all_neighbors.append(get_neighbors(trnrss, i, k))

    return all_neighbors

# predict position of neighbors 
def predict_position(all_neighbors, trncrd, k):
    predicted_pos = []
    
    # iterate all neighbors to predict position. Find k indexes in trncrd and calculate mean value.
    for knn in all_neighbors:    
        x, y, z = 0, 0, 0
        for i in knn:
            x += trncrd[i][0]
            y += trncrd[i][1]
            z += trncrd[i][2]
        predicted_pos.append([x/k, y/k, round(z/k, 1)]) # mean of the error
        
    return predicted_pos

# error calculation: returns euclidean distance between predictions and tstcrd
def calculate_error(predicted_pos, tstcrd):
    error_distances = [math.dist(predicted_pos[i], tstcrd[i]) for i, val in enumerate(predicted_pos)]
    return error_distances

In [9]:
# calculates knn and returns prediction of position
def knn_and_prediction(k, filename):
    trnrss, trncrd, tstrss, tstcrd = load_Data(filename)
    print(tstrss)
    neighbors = knn(k, tstrss, trnrss)
    return predict_position(neighbors, trncrd, k)

# calculates knn, predicts position and returns error distances between prediction and real position
def positioning_error(k, filename):
    trnrss, trncrd, tstrss, tstcrd = load_Data(filename)
    neighbors = knn(k, tstrss, trnrss)
    positions = predict_position(neighbors, trncrd, k)
    return calculate_error(positions, tstcrd)

### Plots

In [5]:
#ECDF function to generate x and y axis data
def ecdf(xdata):
    xdataecdf = np.sort(xdata)
    ydataecdf = np.arange(1, len(xdata) + 1) / len(xdata)
    return xdataecdf, ydataecdf

def plot_ecdf(data1, data2, data3, title, figure):
    #Get the x and y data for ecdf plot from ecdf method
    x1,y1 = ecdf(data1)
    x5,y5 = ecdf(data2)
    x11,y11 = ecdf(data3)

    #Plot the data using matplotlib
    plt.figure(figure)
    plt.plot(x1, y1, marker = '.', linestyle = 'none', markersize = 3)
    plt.plot(x5, y5, marker = '.', linestyle = 'none', markersize = 3)
    plt.plot(x11, y11, marker = '.', linestyle = 'none', markersize = 3)
    plt.legend(("k=1", "k=5", "k=11"))
    plt.title(title)
    plt.xlabel('Error distance')
    plt.ylabel('Error probability')
    plt.margins(0.1)

### Distance metrics

In [6]:
# auxiliar functions
def normalize(h):
    return h / np.sum(h)

def square(n):
    return n*n

def prob_sym(P, Q):
    return 2*squared(P, Q)

def half(n):
    return 0.5*n

def exp32(n):
    return n*1.5

# distance metrics

def gower(P, Q):
    abs_subs = list(map(abs, list(np.subtract(P, Q))))
    return sum(abs_subs)/len(P)

def soergel(P, Q):
    abs_subs = list(map(abs, list(np.subtract(P, Q))))
    max_el = list(map(max, P, Q))
    return sum(abs_subs)/sum(max_el)

def kulczynski_d(P, Q):
    abs_subs = list(map(abs, list(np.subtract(P, Q))))
    min_el = list(map(min, P, Q))
    return sum(abs_subs)/sum(min_el)

def lorentzian(P, Q):
    abs_subs = list(map(abs, list(np.subtract(P, Q))))
    add_1 = list(np.asarray(abs_subs) + 1)
    return sum(np.log(add_1))

def intersection(P, Q):
    return sum(list(map(min, P, Q)))

def wavehedges(P, Q):
    min_el = list(map(min, P, Q))
    subs = list(1 - np.asarray(min_el))
    max_el = list(map(max, P, Q))
    return sum(list(map(truediv, subs, max_el)))

def czekanowski_s(P, Q):
    min_el = list(map(min, P, Q))
    subs = list(1 - np.asarray(min_el))
    sumPQ = list(map(add, P, Q) )
    return 2*sum(list(map(truediv, subs, sumPQ)))

def czekanowski_d(P, Q):
    return 1 - czekanowski_s(P, Q)

def motyka_s(P, Q):
    min_el = list(map(min, P, Q))
    subs = list(1 - np.asarray(min_el))
    sumPQ = list(map(add, P, Q))
    return sum(list(map(truediv, subs, sumPQ)))

def motyka_d(P, Q):
    return 1 - motyka_s(P, Q)

def kulczynski_s(P, Q):
    P = np.array(P)
    Q = np.array(Q)
    abs_subs = list(map(abs, list(np.subtract(P, Q))))
    min_el = list(map(min, P, Q))
    dist = 0
    
    if sum(abs_subs) != 0:
        dist = sum(list(map(truediv, min_el, abs_subs)))
    
    if sum(min_el) != 0:
        dist = 1/dist
    
    return dist

def ruzicka(P, Q):
    min_el = list(map(min, P, Q))
    max_el = list(map(max, P, Q))
    return 1 - sum(min_el)/sum(max_el)
        
def tanimoto(P, Q):
    min_el = list(map(min, P, Q))
    return (sum(P) + sum(Q) - 2*sum(min_el)) /  (sum(P) + sum(Q) - sum(min_el))

def harmonic(P, Q):
    min_el = list(map(min, P, Q))
    multPQ = list(map(mul, P, Q))
    sumPQ = list(map(add, P, Q))
    return 2*sum(list(map(truediv, multPQ, sumPQ)))

def kumar(P, Q):
    return np.dot(P, Q) / (np.dot(P, P) + np.dot(Q, Q) - np.dot(P, Q))

def jaccard_s(P, Q):
    return kumar(P, Q)
        
def jaccard_d(P, Q):
    return 1 - jaccard_s(P, Q)

def dice_s(P, Q):
    return 2*np.dot(P, Q) / (np.dot(P, P) + np.dot(Q, Q))
    
def dice_d(P, Q):
    return 1 - dice_s(P, Q)
    
def fidelity(P, Q):
    multPQ = list(map(mul, P, Q))
    return sum(map(math.sqrt, multPQ))

def bhattacharrya(P, Q):
    multPQ = list(map(mul, P, Q))
    return -np.log(sum(np.sqrt(multPQ)))

def hellinger(P, Q):
    sqrtP = np.sqrt(P)
    sqrtQ = np.sqrt(Q)
    return np.sqrt(2*sum((sqrtP - sqrtQ)**2))

def matusita(P, Q):
    sqrtP = np.sqrt(P)
    sqrtQ = np.sqrt(Q)
    return np.sqrt(np.dot(sqrtP-sqrtQ, sqrtP-sqrtQ))

def squared_chord(P, Q):
    sqrtP = np.sqrt(P)
    sqrtQ = np.sqrt(Q)
    return np.dot(sqrtP-sqrtQ, sqrtP-sqrtQ)

def pearson(P, Q):
    subs = list(np.subtract(P, Q))
    sqr = map(square, subs)
    div = map(truediv, sqr, Q)
    return sum(div)

def neyman(P, Q):
    subs = list(np.subtract(P, Q))
    sqr = map(square, subs)
    div = map(truediv, sqr, P)
    return sum(div)

def squared(P, Q):
    subs = list(np.subtract(P, Q))
    sumPQ = list(map(add, P, Q))
    sqr = map(square, subs)
    div = map(truediv, sqr, sumPQ)
    return sum(div)

def divergence(P, Q):
    subs = list(np.subtract(P, Q))
    sumPQ = list(map(add, P, Q))
    sqr1 = map(square, subs)
    sqr2 = map(square, sumPQ)
    div = map(truediv, sqr1, sqr2)
    return 2*sum(div)
    
def clark(P, Q):
    abs_subs = list(map(abs, list(np.subtract(P, Q))))
    sumPQ = list(map(add, P, Q))
    sqr = map(square, sumPQ)
    div = map(truediv, abs_subs, sqr)
    return np.sqrt(sum(div))

def additive_sym(P, Q):
    subs = list(np.subtract(P, Q))
    sumPQ = list(map(add, P, Q))
    sqr = map(square, subs)
    multPQ = list(map(mul, P, Q))
    mult = list(map(mul, sqr, sumPQ))
    div = map(truediv, mult, multPQ)
    return sum(div)

def kullback_PQ(P, Q):
    div = list(map(truediv, P, Q))
    mult = list(map(mul, P, np.log(div)))
    return sum(mult)

def kullback_QP(P, Q):
    div = list(map(truediv, Q, P))
    mult = list(map(mul, Q, np.log(div)))
    return sum(mult)

def jeffreys(P, Q):
    subs = list(np.subtract(P, Q))
    div = list(map(truediv, P, Q))
    mult = list(map(mul, subs, np.log(div)))
    return sum(mult)
    
def k_divergence(P, Q):
    sumPQ = list(map(add, P, Q))
    mult = list(map(mul, P, sumPQ))
    log = 2*np.log(mult)
    mult2 = list(map(mul, P, log))
    return sum(mult2)
    
def topsoe(P, Q):
    sumPQ = list(map(add, P, Q))
    div1 = list(map(truediv, P, sumPQ))
    div2 = list(map(truediv, Q, sumPQ))
    log1 = np.log(2*div1)
    log2 = np.log(2*div2)
    mult1 = list(map(mul, P, log1))
    mult2 = list(map(mul, Q, log2))
    return sum(mult1 + mult2)
    
def jensen_shannon(P, Q):
    return 0.5*(kullback_PQ(P, Q) + kullback_QP(P, Q))

def jensen_diff(P, Q):
    sumPQ = list(map(add, P, Q))
    mult1 = list(map(mul, P, np.log(P)))
    mult2 = list(map(mul, Q, np.log(Q)))
    log1 = np.log(list(map(half, sumPQ)))
    mult3 = list(map(mul, sumPQ, log1))
    sum2 = list(map(add, mult1, mult2))
    sum3 = list(map(add, sum2, mult3))
    return sum(list(map(half, sum3)))

def taneja(P, Q):
    sumPQ = list(map(add, P, Q))
    half1 = list(map(half, sumPQ))
    mult1 = list(map(mul, half1, np.log(half1)))
    sqrt = np.sqrt(np.dot(P, Q))
    return sum(mult1/sqrt)

def kumar_johnson(P, Q):
    exp1 = list(map(square, P))
    exp2 = list(map(square, Q))
    subs = list(np.subtract(exp1, exp2))
    exp3 = list(map(square, exp2))
    mult1 = list(map(mul, P, Q))
    exp4 = list(map(exp32, mult1))
    div1 = list(map(truediv, exp3, exp4))
    return 0.5*sum(div1)

def avgL(P, Q):
    abs_subs = list(map(abs, list(np.subtract(P, Q))))
    return 0.5*(sum(abs_subs) + max(abs_subs))